In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils
import re

import yaml
import os

In [ ]:
dir_path = '/content/drive/MyDrive/Dataset'
files_list = os.listdir(dir_path + os.sep)

In [ ]:
questions, answers = [], []

for filepath in files_list:
    file_ = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file_)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])

In [ ]:
answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1


In [ ]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

In [ ]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
encoder_input_data = np.array(padded_questions)

In [ ]:
encoder_input_data.shape

(503, 22)

In [ ]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array(padded_answers)

In [ ]:
decoder_input_data.shape

(503, 74)

In [ ]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)

In [ ]:
decoder_output_data.shape

(503, 74, 1794)

In [ ]:
# Embedding, LSTM and Desne layers
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)


decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 22)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 74)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 22, 200)      358800      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 74, 200)      358800      ['input_2[0][0]']                
                                                                                              

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=16, epochs=500) 

Epoch 1/500
32/32 [==============================] - 18s 301ms/step - loss: 1.3783 - accuracy: 0.0896
Epoch 2/500
32/32 [==============================] - 10s 298ms/step - loss: 1.1615 - accuracy: 0.1153
Epoch 3/500
32/32 [==============================] - 11s 360ms/step - loss: 1.1092 - accuracy: 0.1246
Epoch 4/500
32/32 [==============================] - 9s 295ms/step - loss: 1.0745 - accuracy: 0.1474
Epoch 5/500
32/32 [==============================] - 10s 299ms/step - loss: 1.0442 - accuracy: 0.1513
Epoch 6/500
32/32 [==============================] - 11s 360ms/step - loss: 1.0207 - accuracy: 0.1622
Epoch 7/500
32/32 [==============================] - 9s 297ms/step - loss: 0.9987 - accuracy: 0.1754
Epoch 8/500
32/32 [==============================] - 9s 295ms/step - loss: 0.9771 - accuracy: 0.1864
Epoch 9/500
32/32 [==============================] - 10s 298ms/step - loss: 0.9567 - accuracy: 0.1987
Epoch 10/500
32/32 [==============================] - 9s 296ms/step - loss: 0.9369 - 

In [ ]:
def inference():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

def preprocess_input(input_sentence):
    tokens = input_sentence.lower().split()
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word]) 
    return preprocessing.sequence.pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = inference()

In [ ]:
tests = ['You can not move', 'You sound like Data', 'Stupid', 'you are idiot', 'i am going to die']

for i in range(5):
    states_values = enc_model.predict(preprocess_input(tests[i]))
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros((1 , 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c] 
    print(f'Human: {tests[i]}')
    print()
    decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 0s 19ms/step
Human: You can not move

Bot:  i can move through a network easily assuming that i'm given the ability to that is
-------------------------
1/1 [==============================] - 0s 19ms/step
Human: You sound like Data

Bot:  yes i am inspired by commander data's artificial personality
-------------------------
1/1 [==============================] - 0s 22ms/step
Human: Stupid

Bot:  do you feel you think of it
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: you are idiot

Bot:  tell me something that
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: i am going to die

Bot:  i always upload yourself
-------------------------
